In [1]:
from goesaws import *
import datetime
import pandas as pd
import sys

date = str(datetime.date.today())

bucket_name = 'noaa-goes16'
product_name = 'ABI-L2-MCMIPM'
lightning_mapper = 'GLM-L2-LCFA'
yr = 2023
day_of_year =205
hr = 18
minutes = 5

In [2]:
abiprefix = gen_prefix(product=product_name,year = yr, day=day_of_year, hour = hr)
abifiles = gen_fn(bucket=bucket_name,prefix=abiprefix)
abidatasets = [gen_data(key=abifiles[i], bucket = bucket_name) for i in range(0,minutes)]
abiDS = xr.concat(abidatasets,dim = 't')

In [3]:
skyprefix = gen_prefix(product = "ABI-L2-ACMM", year = yr, day = day_of_year, hour = hr)
skyfiles = gen_fn(bucket = bucket_name, prefix=skyprefix)
skydatasets = [gen_data(key=skyfiles[i], bucket = bucket_name) for i in range(0,minutes)]
skyDS = xr.concat(skydatasets,dim = 't')

In [4]:
dataset2_resampled = skyDS.interp(
    x=abiDS["x"].data,
    y=abiDS["y"].data,
    t = abiDS["t"].data,
    method='linear',
    kwargs={'fill_value': "extrapolate"}
)

In [5]:
nfile = xr.merge([abiDS,dataset2_resampled])
nfile = nfile.coarsen(x=4,y=4).mean()
nfile = calc_latlon(nfile)

In [6]:
df = nfile.to_dataframe()
df

CMI_C01  \
t                             y        x         number_of_time_bounds number_of_image_bounds RTM_BT_comparison_bands number_of_focal_planes_used number_of_LZA_bounds number_of_SZA_bounds band             
2023-07-24 18:00:30.771111936 0.119728 -0.013888 0                     0                      0                       0                           0                    0                    0     0.206845   
                                                                                                                                                                       1                    0     0.206845   
                                                                                                                                                  1                    0                    0     0.206845   
                                                                                                                                                                       1                    0     0.206845   
                                                                                                                      1                           0                    0                    0     0.206845   
...                                                                                                                                                                                                    ...   
2023-07-24 18:04:28.161112960 0.091952  0.013888 1                     1                      1                       0                           1                    1                    0     0.152619   
                                                                                                                      1                           0                    0                    0     0.152619   
                                                                                                                                                                       1                    0     0.152619   
                                                                                                                                                  1                    0                    0     0.152619   
                                                                                                                                                                       1                    0     0.152619   

                                                                                                                                                                                                  DQF_C01  \
t                             y        x         number_of_time_bounds number_of_image_bounds RTM_BT_comparison_bands number_of_focal_planes_used number_of_LZA_bounds number_of_SZA_bounds band            
2023-07-24 18:00:30.771111936 0.119728 -0.013888 0                     0                      0                       0                           0                    0                    0         0.0   
                                                                                                                                                                       1                    0         0.0   
                                                                                                                                                  1                    0                    0         0.0   
                                                                                                                                                                       1                    0         0.0   
                                                                                                                      1                           0                    0                    0         0.0   
...                                                                              

In [7]:
glmprefix = gen_prefix(product = "GLM-L2-LCFA", year = yr, day = day_of_year, hour = hr)
glmfiles = gen_fn(bucket = bucket_name, prefix=glmprefix)
#There should be 180 GLM files per hour (one every 20 seconds)
glmdatasets = [gen_data(key=glmfiles[i], bucket = bucket_name) for i in range(0,minutes*3)]
latitudes = [np.array(glmdatasets[i]["event_lat"].data) for i in range(0,minutes*3)]
lats = np.concatenate(latitudes)
longitudes = [np.array(glmdatasets[i]["event_lon"].data) for i in range(0,minutes*3)]
lons= np.concatenate(longitudes)
strikes = list(zip(lats,lons))

In [8]:
glmdatasets[0]

<xarray.Dataset>
Dimensions:                                 (number_of_events: 4946,
                                             number_of_groups: 2181,
                                             number_of_flashes: 173,
                                             number_of_time_bounds: 2,
                                             number_of_wavelength_bounds: 2,
                                             number_of_field_of_view_bounds: 2)
Coordinates: (12/21)
    event_id                                (number_of_events) uint32 ...
    event_time_offset                       (number_of_events) datetime64[ns] ...
    event_lat                               (number_of_events) float64 29.24 ...
    event_lon                               (number_of_events) float64 -80.85...
    event_parent_group_id                   (number_of_events) uint32 ...
    group_id                                (number_of_groups) uint32 ...
    ...                                      ...
    product_time                            datetime64[ns] ...
    lightning_wavelength                    float32 ...
    group_time_threshold                    float32 ...
    flash_time_threshold                    float32 ...
    lat_field_of_view                       float32 ...
    lon_field_of_view                       float32 ...
Dimensions without coordinates: number_of_events, number_of_groups,
                                number_of_flashes, number_of_time_bounds,
                                number_of_wavelength_bounds,
                                number_of_field_of_view_bounds
Data variables: (12/27)
    event_energy                            (number_of_events) float32 ...
    group_frame_time_offset                 (number_of_groups) datetime64[ns] ...
    group_area                              (number_of_groups) float32 ...
    group_energy                            (number_of_groups) float32 ...
    group_quality_flag                      (number_of_groups) float32 ...
    flash_frame_time_offset_of_first_event  (number_of_flashes) datetime64[ns] ...
    ...                                      ...
    nominal_satellite_subpoint_lon          float32 ...
    lon_field_of_view_bounds                (number_of_field_of_view_bounds) float32 ...
    percent_uncorrectable_L0_errors         float32 ...
    algorithm_dynamic_input_data_container  int32 ...
    processing_parm_version_container       int32 ...
    algorithm_product_version_container     int32 ...
Attributes: (12/29)
    production_site:           WCDAS
    featureType:               point
    dataset_name:              OR_GLM-L2-LCFA_G16_s20232051800000_e2023205180...
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    institution:               DOC/NOAA/NESDIS> U.S. Department of Commerce, ...
    ...                        ...
    time_coverage_start:       2023-07-24T18:00:00.0Z
    time_coverage_end:         2023-07-24T18:00:20.0Z
    production_data_source:    Realtime
    production_environment:    OE
    LUT_Filenames:             GLM_CALINR_AllFilters(FM1E_CDRL79RevK_DO_09_00...
    id:                        05e3f19c-2f26-48cb-bab7-4b7f02b9540d

In [9]:
from scipy.spatial import cKDTree
tree = cKDTree(df[['lat', 'lon']].values)
distances, indices = tree.query(strikes)
df = df.reset_index(drop=True)  # reset the index of the df dataframe
lightning_df = pd.DataFrame({
    'strike_lat': [strike[0] for strike in strikes],
    'strike_lon': [strike[1] for strike in strikes],
    'nearest_lat': df.loc[indices, 'lat'].values,
    'nearest_lon': df.loc[indices, 'lon'].values,
    'distance': distances
})

lightning_df["lightning"] = 0
lightning_df.loc[distances < 5, 'lightning'] = 1
lightning_df["Coordinates"] = list(zip(lightning_df["nearest_lat"],lightning_df["nearest_lon"]))


In [10]:
lightning_df

,strike_lat,strike_lon,nearest_lat,nearest_lon,distance,lightning,Coordinates
0,29.235165,-80.851133,32.346390,-80.460047,3.135709,1,"(32.34639017911491, -80.4600470588864)"
1,29.157977,-80.845040,32.346390,-80.460047,3.211573,1,"(32.34639017911491, -80.4600470588864)"
2,29.157977,-80.845040,32.346390,-80.460047,3.211573,1,"(32.34639017911491, -80.4600470588864)"
3,29.235165,-80.851133,32.346390,-80.460047,3.135709,1,"(32.34639017911491, -80.4600470588864)"
4,29.157977,-80.845040,32.346390,-80.460047,3.211573,1,"(32.34639017911491, -80.4600470588864)"
...,...,...,...,...,...,...,...
78343,53.362709,-61.594599,45.661993,-68.211582,10.153102,0,"(45.661992651777275, -68.21158155470641)"
78344,53.370834,-61.456472,45.661993,-68.211582,10.249768,0,"(45.661992651777275, -68.21158155470641)"
78345,16.580291,-101.271591,32.346390,-80.460047,26.109198,0,"(32.34639017911491, -80.4600470588864)"
78346,16.582323,-101.354873,32.346390,-80.460047,26.174407,0,"(32.34639017911491, -80.4600470588864)"


In [11]:
features = ["CMI_C01", "CMI_C02","CMI_C03","CMI_C04","CMI_C05","CMI_C06","CMI_C07","CMI_C08","CMI_C09","CMI_C10","CMI_C11","CMI_C12","CMI_C13","CMI_C14","CMI_C15","CMI_C16","ACM", "BCM", "Cloud_Probabilities","lat","lon","Coordinates","Lightning"]
df["Coordinates"] = list(zip(df["lat"],df["lon"]))



In [12]:

df["Lightning"] = df["Coordinates"].isin(lightning_df["Coordinates"])
nmap = {True:1, False:0}
df["Lightning"] = df["Lightning"].map(nmap)
subset = df[features]
subset
    

,CMI_C01,CMI_C02,CMI_C03,CMI_C04,CMI_C05,CMI_C06,CMI_C07,CMI_C08,CMI_C09,CMI_C10,...,CMI_C14,CMI_C15,CMI_C16,ACM,BCM,Cloud_Probabilities,lat,lon,Coordinates,Lightning
0,0.206845,0.118551,0.253968,0.005159,0.129127,0.067619,295.914398,236.170013,243.878540,252.028976,...,289.081482,285.052429,267.335999,0.562537,0.187515,0.216501,45.661993,-81.788418,"(45.66199265221488, -81.78841847220028)",1
1,0.206845,0.118551,0.253968,0.005159,0.129127,0.067619,295.914398,236.170013,243.878540,252.028976,...,289.081482,285.052429,267.335999,0.562537,0.187515,0.216501,45.661993,-81.788418,"(45.66199265221488, -81.78841847220028)",1
2,0.206845,0.118551,0.253968,0.005159,0.129127,0.067619,295.914398,236.170013,243.878540,252.028976,...,289.081482,285.052429,267.335999,0.562537,0.187515,0.216501,45.661993,-81.788418,"(45.66199265221488, -81.78841847220028)",1
3,0.206845,0.118551,0.253968,0.005159,0.129127,0.067619,295.914398,236.170013,243.878540,252.028976,...,289.081482,285.052429,267.335999,0.562537,0.187515,0.216501,45.661993,-81.788418,"(45.66199265221488, -81.78841847220028)",1
4,0.206845,0.118551,0.253968,0.005159,0.129127,0.067619,295.914398,236.170013,243.878540,252.028976,...,289.081482,285.052429,267.335999,0.562537,0.187515,0.216501,45.661993,-81.788418,"(45.66199265221488, -81.78841847220028)",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,0.152619,0.064067,0.051528,0.003968,0.039901,0.030317,298.689178,235.731674,244.288712,254.545273,...,291.913177,288.116089,272.375946,0.999994,0.187493,0.288797,32.346390,-69.539953,"(32.34639017891283, -69.53995296267827)",1
4999996,0.152619,0.064067,0.051528,0.003968,0.039901,0.030317,298.689178,235.731674,244.288712,254.545273,...,291.913177,288.116089,272.375946,0.999994,0.187493,0.288797,32.346390,-69.539953,"(32.34639017891283, -69.53995296267827)",1
4999997,0.152619,0.064067,0.051528,0.003968,0.039901,0.030317,298.689178,235.731674,244.288712,254.545273,...,291.913177,288.116089,272.375946,0.999994,0.187493,0.288797,32.346390,-69.539953,"(32.34639017891283, -69.53995296267827)",1
4999998,0.152619,0.064067,0.051528,0.003968,0.039901,0.030317,298.689178,235.731674,244.288712,254.545273,...,291.913177,288.116089,272.375946,0.999994,0.187493,0.288797,32.346390,-69.539953,"(32.34639017891283, -69.53995296267827)",1


In [13]:
subset["Lightning"].value_counts()

0    4819520
1     180480
Name: Lightning, dtype: int64

Right now, all of the lightning strikes might not necessarily be close to the assigned value. Also we have to deal with the time component. But this is much better than before. 

In [14]:
#subset.to_csv("/Users/robbiefeldstein/Documents/Programming/Research/Datasets/subset.csv")